In [3]:
import requests
from bs4 import BeautifulSoup
import re
from time import sleep
import random
from __future__ import print_function
import pickle
import pandas as pd
import sqlite3

In [25]:
# Search for all of the possible permutations of data science jobs
# Pull from Indeed.com

search_list = ['data+analyst','business+analyst','data+scientist','data+engineer','data+architect']

job_ids = []

for title in search_list:
    
    print('Parsing {} '.format(title.replace('+',' ')), end='')
    
    test = "https://www.indeed.com.sg/jobs?q={}&l=Singapore&jt=fulltime&start=".format(title)
        
    sleep(random.randint(10, 20))
    
    for page in range(110,200,10): # Change this to make it more managable
        
        #print('Parsing page {}'.format(page))
        
        url = test + '{}'.format(page)
        
        # make request for that page
        sauce = requests.get(url).text
    
        # turn into a BeautifulSoup object
        soup = BeautifulSoup(sauce, 'lxml')
    
        # get the job IDs
        for listing in soup.find_all('a', {'href': re.compile(r'jk\=(\w+)')}):
            job_id = re.findall(r'jk\=(\w+)', listing['href'])[0]
            job_ids.append(job_id)
    
        sleep(random.randint(3, 15))
        
        print('{}, '.format(page), end='')

Parsing data analyst 110, 120, 130, 140, 150, 160, 170, 180, 190, Parsing business analyst 110, 120, 130, 140, 150, 160, 170, 180, 190, Parsing data scientist 110, 120, 130, 140, 150, 160, 170, 180, 190, Parsing data engineer 110, 120, 130, 140, 150, 160, 170, 180, 190, Parsing data architect 110, 120, 130, 140, 150, 160, 170, 180, 190, 

In [26]:
# Remove all duplicates
job_ids = list(set(job_ids))

In [27]:
len(job_ids)

315

In [ ]:
# Pickle the job ids
# pickle.dump(job_ids, open('job_ids_1604.p','wb'))

In [ ]:
# Unpickle the job ids
# pickle_off = open('job_ids_1604.p','rb')
# job_ids = pickle.load(pickle_off)

In [ ]:
# len(job_ids)

In [28]:
job_ids[:5]

['793ad3c91a37b14f',
 '83397882e381e95f',
 '9f10aaa352412827',
 'fbf5e64adfd30998',
 '6b5b2314662bb16d']

In [29]:
titles = []
company_name = []
job_descriptions = []

#no_data = 'NA'

for id in job_ids:
    
    url = "https://www.indeed.com.sg/viewjob?jk={}".format(id)
    
    sauce = requests.get(url).text
    soup = BeautifulSoup(sauce, 'lxml')
    
    print('--', end='')
    
    sleep(random.randint(2, 5)) # Avoiding the anti-bot mechanism

    for i in soup.find_all('table', {'id': 'job-content'}):
        
        # Extract Job Titles
        try:
            job_title = i.find('b', {'class': 'jobtitle'}).font.text
            titles.append(job_title)
        except Exception:
            pass
        
        # Extract Company Name
        try:
            company = i.find('span', {'class': 'company'}).text
            company_name.append(company)
        except Exception:
            pass
    
        # Extract Job Description
        try:
            job_description = i.find('span', {'class':'summary'}).text.strip()
            job_descriptions.append(job_description)
        except Exception:
            pass
    
        #sleep(random.randint(2, 5)) # Avoiding the anti-bot mechanism
    
    print('>', end='')

-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->

In [30]:
print('Total number of items found: {}'.format(len(titles)))

Total number of items found: 315


In [31]:
print('Total number of items found: {}'.format(len(company_name)))

Total number of items found: 315


In [32]:
print('Total number of items found: {}'.format(len(job_descriptions)))

Total number of items found: 315


In [33]:
titles[-5:]

[u'Procurement Analyst',
 u'SOFTWARE ENGINEER',
 u'Analytical Scientist',
 u'Senior Business Analyst',
 u'ETL Lead']

In [34]:
job_ids[-5:]

['2a69f2fc39014ede',
 '809aeaff8651dc59',
 'f7c2497171d20429',
 '48fbd9c3afbd9bf8',
 '497f563077c23137']

In [35]:
# DataFrame
df = pd.DataFrame({'id': job_ids,'title': titles,'description': job_descriptions})

In [36]:
df.head()

,description,id,title
0,"Backed by Benchmark, Index Ventures, and Sequo...",793ad3c91a37b14f,Sales Engineer - Singapore
1,Demand Center Operations Analyst (12 mths Fixe...,83397882e381e95f,Demand Center Operations Analyst (12 mths Fixe...
2,Roles & Responsibilities\nKey responsibilities...,9f10aaa352412827,Data Scientist
3,Responsibilities:\n You will work in the Plann...,fbf5e64adfd30998,Operations Planning Analyst (Data Analytics)
4,"Risk Data Analyst, Group Portfolio Analytics &...",6b5b2314662bb16d,"Risk Data Analyst, Group Portfolio Analytics &..."


In [37]:
df['id'].duplicated(keep='first').sum()

0

In [ ]:
# df.drop_duplicates(subset='id', keep='first', inplace=True)

In [38]:
# # Check against already mined IDs
mined_ids = pickle.load(open('mined_ids.p','rb'))

In [39]:
# # Check against already scraped data
#
df = df[~df['id'].isin(mined_ids)]

In [40]:
len(df)

138

In [44]:
# Saving to csv instead
df.to_csv('scraped_1604_2.csv', encoding='utf-8', index=False)

In [41]:
# # Saving to a SQL file
connection = sqlite3.connect('job_scraped.db.sqlite')
df.to_sql(name = 'jobs', con = connection, if_exists = 'append', index = False)

In [ ]:
# len(mined_ids)

In [42]:
# Add latest mined job_ids into the mined_job_ids
mined_ids.extend(df['id'].tolist())

In [ ]:
# len(mined_ids)

In [43]:
# Pickle the job ids
pickle.dump(mined_ids, open('mined_ids.p','wb'))